# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.847757361226                   -0.70    4.5   88.4ms
  2   -7.852402901177       -2.33       -1.53    1.0   18.5ms
  3   -7.852610283802       -3.68       -2.54    1.2   19.3ms
  4   -7.852645471192       -4.45       -2.78    2.5   25.5ms
  5   -7.852646095778       -6.20       -2.88    1.0   18.4ms
  6   -7.852646665939       -6.24       -3.84    1.0   18.6ms
  7   -7.852646684816       -7.72       -4.64    1.5   20.9ms
  8   -7.852646686680       -8.73       -5.03    1.8   22.1ms
  9   -7.852646686726      -10.34       -5.93    1.0   18.7ms
 10   -7.852646686728      -11.53       -5.71    2.2   25.1ms
 11   -7.852646686730      -11.95       -6.09    1.0   18.8ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.847790981647                   -0.70           4.5    389ms
  2   -7.852561570870       -2.32       -1.62   0.80    2.0    2.23s
  3   -7.852641258679       -4.10       -2.70   0.80    1.0    228ms
  4   -7.852646519488       -5.28       -3.40   0.80    2.0   22.0ms
  5   -7.852646682183       -6.79       -4.42   0.80    1.5   49.0ms
  6   -7.852646686634       -8.35       -4.86   0.80    2.0   23.3ms
  7   -7.852646686727      -10.04       -5.52   0.80    1.0   17.4ms
  8   -7.852646686730      -11.52       -6.51   0.80    1.5   19.6ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +1.353445736983                   -0.98    4.21s
  2   -0.979628807200        0.37       -0.65    130ms
  3   -3.515372575927        0.40       -0.45   33.2ms
  4   -4.472381619054       -0.02       -0.61   33.1ms
  5   -6.324077027387        0.27       -0.70   33.2ms
  6   -7.082503819583       -0.12       -1.07   33.3ms
  7   -7.422128339679       -0.47       -1.47   25.3ms
  8   -7.636194925183       -0.67       -1.78   25.1ms
  9   -7.691531098159       -1.26       -1.98   25.1ms
 10   -7.732583499018       -1.39       -2.02   25.2ms
 11   -7.764904125409       -1.49       -2.16   33.6ms
 12   -7.786913329779       -1.66       -2.45   25.3ms
 13   -7.801757004899       -1.83       -2.53   25.2ms
 14   -7.819359780326       -1.75       -2.24   25.1ms
 15   -7.840327809071       -1.68       -1.93   51.0ms
 16   -7.846586255450       -2.20       -2.44   25.9ms
 17   -7.85

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.847811815987                   -0.70    4.8   36.3ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645852653                   -1.63    13.8s
  2   -7.852646686730       -6.08       -3.69    3.56s
  3   -7.852646686730      -13.26       -7.20    110ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 8.033246924295935e-7
|ρ_newton - ρ_scfv| = 5.213174397767181e-7
|ρ_newton - ρ_dm|   = 4.0404337849209055e-9
